In [27]:
from copy import deepcopy
import itertools

class OrderManager:
    def __init__(self):
        # you will define the data structure you need to answer this problem
        # you will certainly have a counter for the order id that you will generate with Order Manager
        self.orderbook = []
        self.buyorders = []
        self.sellorders = []
        self.ordercounter = 0
        self.buyvolume = 0
        self.sellvolume = 0
        self.filled = 0
        self.positions = 0
        self.holdings = 0
        
    def give_id(self):
        return lambda x=itertools.count(): next(x)

    def handle_order_from_ts(self,o):
        # you will certainly generate a new order id and attached this order id to the order o
        # you will certainly define a state new for this order o
        # you will certainly append this order to the data structure
        o = deepcopy(o)
        counter = self.ordercounter
        o['id'] = counter +1
        o['state'] = 'new'
        self.ordercounter += 1
        self.orderbook.append(o)
        if o['side'] == "buy":
            self.buyvolume += o['quantity'] * o['price']
            self.buyorders.append(o)
        else:
            self.sellvolume += o['quantity'] * o['price']
            self.sellorders.append(o)
            
    def handle_order_from_market(self,o):
        # you will handle the order response o
        # you will modify the state of the orders in the data structure
        found = 0
        print(self.orderbook)
        for i in self.orderbook:
            if i['id'] == o['id']:
                found = 1
                i['state'] = o['state']
                if o['state'] == 'filled':
                    x = o['price'] * o['quantity']
                    self.holdings += deepcopy(x)
                    self.filled += 1
                    if o['side'] == "buy":
                        y = deepcopy(o['quantity'])
                        self.positions += y
                    elif o['side'] == 'sell':
                        z = deepcopy(o['quantity'])
                        self.positions -= z
                        
                break
        if not found:
            print("order id {} not found".format(o['id']))
    def get_positions(self):
        # you will return the position
        return self.positions
    def get_holdings(self):
        # you will return the holdings
        return self.holdings
    def get_total_filled(self):
        #you will return the number of filled orders
        return self.filled
    def get_unacknowledged_orders(self):
        # you will follow the instructions above
        return sum([1 for i in self.orderbook if i['state'] == 'new'])
    
    def get_unacknowledged_buy_volume(self):
        # you will follow the instructions above
        return self.buyvolume
    
    def get_unacknowledged_sell_volume(self):
        # you will follow the instructions above
        return self.sellvolume



In [28]:
def test1(): #0
    om=OrderManager()
    o1={'quantity' : 100, 'price' : 10, 'side' : 'buy'}
    om.handle_order_from_ts(o1)
    print('New order' + str(om.get_unacknowledged_orders()))
    o1['quantity'] = 5
    o1['price']=5
    om.handle_order_from_ts(o1)
    print('New order' + str(om.get_unacknowledged_orders()))
    print('Unacknowledged volume for buy in $:' + str(om.get_unacknowledged_buy_volume()))
    print('Unacknowledged volume for sell in $:' + str(om.get_unacknowledged_sell_volume()))
    o1['side'] = 'sell'
    om.handle_order_from_ts(o1)
    print('Unacknowledged volume for buy in $:' + str(om.get_unacknowledged_buy_volume()))
    print('Unacknowledged volume for sell in $:' + str(om.get_unacknowledged_sell_volume()))
    print('Unacknowledged #:' + str(om.get_unacknowledged_orders()))

    return om



In [29]:
test1()

New order1
New order2
Unacknowledged volume for buy in $:1025
Unacknowledged volume for sell in $:0
Unacknowledged volume for buy in $:1025
Unacknowledged volume for sell in $:25
Unacknowledged #:3


In [30]:
def test2(): #1
    om=test1()
    o1 = {'quantity': 100, 'price': 10, 'side': 'buy' , 'id' : 1 , 'state' : 'filled'}
    om.handle_order_from_market(o1)
    print('Number of fills:' + str(om.get_total_filled()))
    o1['id']=2
    o1['state']='cancelled'
    om.handle_order_from_market(o1)
    print('Number of fills:' + str(om.get_total_filled()))
    print('Unacknowledged #:' + str(om.get_unacknowledged_orders()))
    o1['id'] = 3
    o1['state'] = 'rejected'
    om.handle_order_from_market(o1)
    print('Number of fills:' + str(om.get_total_filled()))
    print('Unacknowledged #:' + str(om.get_unacknowledged_orders()))

In [31]:
test2()

New order1
New order2
Unacknowledged volume for buy in $:1025
Unacknowledged volume for sell in $:0
Unacknowledged volume for buy in $:1025
Unacknowledged volume for sell in $:25
Unacknowledged #:3
[{'quantity': 100, 'price': 10, 'side': 'buy', 'id': 1, 'state': 'new'}, {'quantity': 5, 'price': 5, 'side': 'buy', 'id': 2, 'state': 'new'}, {'quantity': 5, 'price': 5, 'side': 'sell', 'id': 3, 'state': 'new'}]
Number of fills:1
[{'quantity': 100, 'price': 10, 'side': 'buy', 'id': 1, 'state': 'filled'}, {'quantity': 5, 'price': 5, 'side': 'buy', 'id': 2, 'state': 'new'}, {'quantity': 5, 'price': 5, 'side': 'sell', 'id': 3, 'state': 'new'}]
Number of fills:1
Unacknowledged #:1
[{'quantity': 100, 'price': 10, 'side': 'buy', 'id': 1, 'state': 'filled'}, {'quantity': 5, 'price': 5, 'side': 'buy', 'id': 2, 'state': 'cancelled'}, {'quantity': 5, 'price': 5, 'side': 'sell', 'id': 3, 'state': 'new'}]
Number of fills:1
Unacknowledged #:0


In [32]:
def test3(): #2
    om=test1()
    o1 = {'quantity': 100, 'price': 10, 'side': 'buy' , 'id' : 4 , 'state' : 'filled'}
    om.handle_order_from_market(o1)

In [33]:
test3()

New order1
New order2
Unacknowledged volume for buy in $:1025
Unacknowledged volume for sell in $:0
Unacknowledged volume for buy in $:1025
Unacknowledged volume for sell in $:25
Unacknowledged #:3
[{'quantity': 100, 'price': 10, 'side': 'buy', 'id': 1, 'state': 'new'}, {'quantity': 5, 'price': 5, 'side': 'buy', 'id': 2, 'state': 'new'}, {'quantity': 5, 'price': 5, 'side': 'sell', 'id': 3, 'state': 'new'}]
order id 4 not found


In [40]:
def test4(): #3
    om=test1()
    o1 = {'quantity': 100, 'price': 10, 'side': 'buy' , 'id' : 1 , 'state' : 'filled'}
    om.handle_order_from_market(o1)
    print('Number of fills:' + str(om.get_total_filled()))
    print('Total position:' + str(om.get_positions()))
    print('Total holdings in $' + str(om.get_holdings()))
    o1['id']=3
    o1['state']='filled'
    om.handle_order_from_market(o1)
    print('Number of fills:' + str(om.get_total_filled()))
    print('Total position:' + str(om.get_positions()))
    print('Total holdings in $' + str(om.get_holdings()))

In [41]:
test4()

New order1
New order2
Unacknowledged volume for buy in $:1025
Unacknowledged volume for sell in $:0
Unacknowledged volume for buy in $:1025
Unacknowledged volume for sell in $:25
Unacknowledged #:3
[{'quantity': 100, 'price': 10, 'side': 'buy', 'id': 1, 'state': 'new'}, {'quantity': 5, 'price': 5, 'side': 'buy', 'id': 2, 'state': 'new'}, {'quantity': 5, 'price': 5, 'side': 'sell', 'id': 3, 'state': 'new'}]
Number of fills:1
Total position:100
Total holdings in $1000
[{'quantity': 100, 'price': 10, 'side': 'buy', 'id': 1, 'state': 'filled'}, {'quantity': 5, 'price': 5, 'side': 'buy', 'id': 2, 'state': 'new'}, {'quantity': 5, 'price': 5, 'side': 'sell', 'id': 3, 'state': 'new'}]
Number of fills:2
Total position:200
Total holdings in $2000
